In [ ]:
import json
with open('properties.json', 'r') as f:
    properties = json.load(f)
properties

In [ ]:
brokers = '{}-cp-kafka:9092'.format(properties['pnda-helm-release'])
topic = properties['topic']

In [ ]:
import random
import time
from kafka import KafkaProducer


def openmetrics_producer(stop_event,brokers, topic, metric, rate):
    producer = KafkaProducer(bootstrap_servers=brokers.split())
    print("Starting Streaming {} to Kafka: rate {} seconds.".format(metric,rate))
    while not stop_event.wait(rate):
        openmetric = '{metric} {value:.2f}'.format(metric=metric, value=random.uniform(0,50))
        future = producer.send(
            topic,
            key=metric.encode('utf-8'),
            value= openmetric.encode('utf-8'))
        # Block for 'synchronous sends'
        record_metadata = future.get(timeout=1)

In [ ]:
import threading

jobs = []
kill_event = threading.Event()
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'humidity{"room"="bathroom"}',1]))
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'temperature{"room"="kitchen"}',0.8]))
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'humidity{"room"="kitchen"}',2.3]))
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'temperature{"room"="corridor"}',1.3]))
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'humidity{"room"="corridor"}',2]))
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'temperature{"room"="bathroom"}',1.6]))


list(map(lambda x: x.start(), jobs))

try:
    while True:
        pass
except KeyboardInterrupt:
    print("stopping openmetrics generators")
    kill_event.set()
    list(map(lambda x: x.join(), jobs))
 
